In [1]:
from NaverCrawling import NaverCrawling
import os
import csv
import json
import time
import pandas as pd
import urllib.request
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import concurrent.futures
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
from functools import partial

client_id = ""
client_secret = ""

C:\Users\art\anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\art\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\art\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Class 호출
crawler = NaverCrawling(client_id, client_secret)

In [5]:
# 폴더 경로 설정
folder_path = './Test/'

# 폴더 내의 파일 리스트 가져오기
file_names = os.listdir(folder_path)

# 첫 번째 파일 이름 추출
first_file_name = file_names[0].split('.')[0].split()[0]

# 결과 파일 경로 설정
result_file = f'./Test/{first_file_name}.csv'

# CSV 파일에서 데이터를 모두 가져오는 함수
def get_all_data_from_csv(folder_path):
    """
    폴더 내의 모든 CSV 파일에서 데이터를 추출하는 함수입니다.
    :param folder_path: CSV 파일이 있는 폴더 경로
    :return: 추출된 데이터를 담은 리스트
    """
    data = []
    csv_files = glob.glob(folder_path + '/*.csv')
    for csv_file in csv_files:
        with open(csv_file, 'r', encoding='utf-8-sig') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                data.append(row)
    return data

# 데이터를 그룹으로 나누어 CSV 파일로 저장하는 함수
def save_data_to_csv_groups(data, group_count):
    """
    데이터를 그룹으로 나누어 CSV 파일로 저장하는 함수입니다.
    :param data: 저장할 데이터를 담은 리스트
    :param group_count: 그룹 개수
    """
    for i in range(group_count):
        group_data = data[i::group_count]
        with open(f'./TEST/group_{i+1}.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
            fieldnames = data[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(group_data)

def delete_rows(csv_file, links):
    df = pd.read_csv(csv_file)
    df = df[~df['link'].isin(links)]
    df.to_csv(csv_file, index=False, encoding='utf-8-sig')

# 폴더 내의 모든 CSV 파일에서 데이터 추출
data = get_all_data_from_csv(folder_path)

# 그룹 개수 설정
group_count = 1

# 데이터를 그룹으로 나누어 CSV 파일로 저장
save_data_to_csv_groups(data, group_count)

ValueError: dict contains fields not in fieldnames: 'content'

In [6]:
min_delay = 1
max_delay = 5
start_time = time.time()
# 병렬로 크롤링 작업 실행
# with ThreadPoolExecutor() as executor:
#     futures = []
#     for i in range(group_count):
#         csv_file = f'./TEST/group_{i+1}.csv'
#         delay = random.uniform(min_delay, max_delay)  # 요청 간격 랜덤 설정
#         user_agent = random.choice(user_agents)  # 사용자 에이전트 랜덤 선택
#         time.sleep(delay)  # 요청 간격 대기
#         futures.append(executor.submit(crawler.crawl_links, csv_file, result_file, user_agent))
    
#     for future in as_completed(futures):
#         pass

# 병렬 크롤링
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(group_count):
        csv_file = f'./TEST/group_{i+1}.csv'
        delay = random.uniform(min_delay, max_delay)
        time.sleep(delay)
        futures.append(executor.submit(crawler.crawl_links, csv_file, result_file, i))
    
    for future in concurrent.futures.as_completed(futures):
        pass   

# 크롤링 결과에서 필요한 열 추출
#crawler.extract_columns(csv_file)

# 총 실행 시간 계산
execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)
hours, minutes = divmod(minutes, 60)

# 총 실행 시간을 시, 분, 초로 변환하여 출력
print(f"Total execution time: {int(hours)}시 {int(minutes)}분 {int(seconds)}초")

Crawling: 100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.48s/link]


크롤링 완료: 2 링크에 대한 본문 내용 추출됨.
Total execution time: 0시 0분 13초


In [5]:
crawler.extract_columns(result_file)

데이터 정제 시작... 불필요한 열 제거 중...
데이터 정제 및 불필요한 열 제거 완료


In [6]:
result_file = './TEST/test.csv'
result_file 

'./TEST/test.csv'

In [12]:
df = pd.read_csv(result_file, encoding='utf-8-sig')
df = df[df['content'].str.strip() != '']
word_list = '개인사업자|행정구역'
df = df[~df['content'].str.contains(word_list)]

df.to_csv(result_file+'2', encoding='utf-8-sig', index=False)

In [14]:
# folder_path = './link'
# csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# for i, file in enumerate(csv_files, start=1):
#     file_path = os.path.join(folder_path, file)
#     df = pd.read_csv(file_path)
    
#     # 파일 이름
#     file_name = os.path.splitext(file)[0]
    
#     # "_link" 제거한 새로운 파일 이름
#     new_file_name = file_name.replace('_links', '')
    
#     # "시도"와 "시군구" 컬럼에 입력
#     sido, sigungu = new_file_name.split(' ', 1)
#     df['시도'] = sido
#     df['시군구'] = sigungu
    
#     # 변경된 DataFrame을 새로운 파일로 저장
#     new_file_path = os.path.join(folder_path, f'{new_file_name}.csv')
#     df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
    
#     # 기존 파일 삭제
#     os.remove(file_path)

In [18]:
# # 현재 폴더의 파일 목록 가져오기
# current_path = "./link"  # 현재 폴더의 경로 지정
# file_list = [file for file in os.listdir(current_path) if os.path.isfile(os.path.join(current_path, file))]

# # 확장자를 제외한 파일명 추출
# file_names = [os.path.splitext(file)[0] for file in file_list]

# # CODE.csv 파일 불러오기
# code_df = pd.read_csv("CODE.csv", encoding='cp949')  # CODE.csv 파일 경로를 지정해주세요

# # "시도 시군구"와 일치하는 파일에 "code" 열 추가하여 "법정동코드" 값 할당
# for file_name in file_names:
#     matching_rows = code_df[code_df["시도 시군구"] == file_name]
#     if not matching_rows.empty:
#         file_path = os.path.join(current_path, file_name + ".csv")
#         df = pd.read_csv(file_path, encoding='utf-8-sig')
#         df["code"] = matching_rows.iloc[0]["법정동코드"]
#         df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [15]:
# df = pd.read_csv('./TEST/제주특별자치도.csv', encoding='utf-8-sig')
# # "시군구" 열 값에 따라 "code" 열 값 설정
# df['code'] = df['시군구'].map({'제주시': '50110', '서귀포시': '50130'})

# # 결과를 파일에 저장
# df.to_csv('제주특별자치도.csv', index=False, encoding='utf-8-sig')
# #crawler.extract_columns('./TEST/제주특별자치도.csv')

In [ ]:
# # 폴더 내 모든 CSV 파일에서 'link' 열의 값만을 리스트로 추출
# def get_all_data_from_csv(folder_path):
#     urls = []
#     csv_files = glob.glob(folder_path + '/*.csv')
#     for csv_file in csv_files:
#         with open(csv_file, 'r', encoding='utf-8-sig') as csvfile:
#             reader = csv.DictReader(csvfile)
#             for row in reader:
#                 urls.append(row['link'])
#     return urls

# # 각 그룹별로 병렬 크롤링 작업 실행
# # csv 파일을 읽어 링크를 crawler.crawling에게 전달.
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # 각 그룹의 CSV 파일을 읽어와서 크롤링 작업 수행
#     futures = []
#     for i in range(group_count):
#         csv_file = f'./TEST/group_{i+1}.csv'
#         group_urls = read_url_group(csv_file)
#         futures.extend(executor.submit(crawler.crawling, link, result_file) for link in group_urls)

#     # 각 크롤링 작업이 완료될 때까지 대기
#     for future in concurrent.futures.as_completed(futures):
#         # 크롤링 작업의 결과나 예외를 가져올 수 있음
#         # result = future.result()
#         pass
    
# get_all_data_from_csv에서 가져온 딕셔너리가 담긴 리스트를 csv로 변환
# def save_url_group(url_list, group_count):
#     for i in range(group_count):
#         group_urls = url_list[i::group_count]  # 그룹별로 URL 선택
#         with open(f'./TEST/group_{i+1}.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
#             writer = csv.writer(csvfile)
#             writer.writerow(['link'])  # 헤더 행 추가
#             for url in group_urls:
#                 writer.writerow([url])  # URL 데이터 작성